In [239]:
import scipy
n = int(1E1)
h = 1./(n+1)
b_tilde = h**2 * 100*np.exp(-10*np.linspace(0,1,n+2))[1:-1]

# 1b. General solution

In [227]:
# Diagonals
a = np.random.rand(n)
b = np.random.rand(n)
c = np.random.rand(n)

In [228]:
# Solution from library function
A = scipy.sparse.diags([list(a[1:]), list(b), list(c[:-1])], [-1,0,1]).toarray()

In [229]:
%%time
x_lib = np.linalg.solve(A, b_tilde)

CPU times: user 16.7 s, sys: 396 ms, total: 17.1 s
Wall time: 4.27 s


In [230]:
# My implementation
b_mod = np.copy(b)
b_tilde_solution = np.copy(b_tilde)

In [231]:
%%time

# Forward substitution
for i in range(1,n):
    f = a[i]/b_mod[i-1]
    b_mod[i] -= f*c[i-1]
    b_tilde_solution[i] -= f*b_tilde_solution[i-1]

# Solving n-1'th x 
b_tilde_solution[n-1] /= b_mod[n-1]

# Backward substitution
for i in range(n-2, -1, -1):
    b_tilde_solution[i] = (b_tilde_solution[i] - c[i]*b_tilde_solution[i+1])/b_mod[i]

CPU times: user 159 ms, sys: 3.45 ms, total: 163 ms
Wall time: 40.6 ms


In [232]:
# Controlling result
assert max(x_lib-b_tilde_solution) < 1E-10


# 1c. Specific solution for 2nd derivative

In [233]:
# Diagonals
a = np.zeros(n)-1
b = np.zeros(n)+2
c = np.zeros(n)-1

In [234]:
# Solution from library function
A = scipy.sparse.diags([list(a[1:]), list(b), list(c[:-1])], [-1,0,1]).toarray()

In [235]:
%%time
x_lib = np.linalg.solve(A, b_tilde)

CPU times: user 17.5 s, sys: 367 ms, total: 17.9 s
Wall time: 4.47 s


In [236]:
# My implementation
b_tilde_solution = np.copy(b_tilde)

In [237]:
%%time

# Forward substitution of b_tilde only
for i in range(1,n):
    b_tilde_solution[i] = (i+1)*b_tilde_solution[i] + b_tilde_solution[i-1]

# Solving n-1'th x
b_tilde_solution[n-1] /= (n+1)

# Backward substitution is with known factors
for i in range(n-2, -1, -1):
    b_tilde_solution[i] = (b_tilde_solution[i] + (i+1)*b_tilde_solution[i+1])/(i+2)

CPU times: user 119 ms, sys: 0 ns, total: 119 ms
Wall time: 29.8 ms


In [238]:
# Controlling result
assert max(x_lib-b_tilde_solution) < 1E-10